# Climbing logs: exploration

Exploration of the usability of the logs

### 0. Variables and dependencies

In [ ]:
import pandas as pd
import numpy as np
import datetime

import matplotlib.pyplot as plt

In [ ]:
from climb.extraction import extract_climbinglogs, extract_sessionlogs
from climb.aggregation import compute_gradepyramid_basic, get_maxgrade_flash, get_mediangrade, compute_gradepyramid
from climb.diagram import plot_doublegradepyramid, plot_gradepyramid

In [ ]:
filein = './data/climbinglogs.xlsx'

### 1. Read in the data

In [1]:
from climb.grade import import_gradestable, create_grademap
grades = import_gradestable()

In [2]:
create_grademap()

{'id2french': {1: '1',
  2: '2',
  3: '2',
  4: '3',
  5: '4a',
  6: '4b',
  7: '4c',
  8: '5a',
  9: '5b',
  10: '5c',
  11: '6a',
  12: '6a+',
  13: '6b',
  14: '6b+',
  15: '6c',
  16: '6c+',
  17: '6c+',
  18: '7a',
  19: '7a+',
  20: '7b',
  21: '7b+',
  22: '7c',
  23: '7c+',
  24: '8a',
  25: '8a+',
  26: '8b',
  27: '8b+',
  28: '8c'},
 'id2usa': {1: '5.0',
  2: '5.1',
  3: '5.2',
  4: '5.3',
  5: '5.4',
  6: '5.5',
  7: '5.6',
  8: '5.7',
  9: '5.8',
  10: '5.9',
  11: '5.10a',
  12: '5.10b',
  13: '5.10c',
  14: '5.10d',
  15: '5.11a',
  16: '5.11b',
  17: '5.11c',
  18: '5.11d',
  19: '5.12a',
  20: '5.12b',
  21: '5.12c',
  22: '5.12d',
  23: '5.13a',
  24: '5.13b',
  25: '5.13c',
  26: '5.13d',
  27: '5.14a',
  28: '5.14b'},
 'id2v-bouldering': {1: 'VB',
  2: 'VB',
  3: 'VB',
  4: 'VB',
  5: 'V0',
  6: 'V0',
  7: 'V0+',
  8: 'V1',
  9: 'V1',
  10: 'V2',
  11: 'V3',
  12: 'V3',
  13: 'V4',
  14: 'V4',
  15: 'V5',
  16: 'V5',
  17: 'V5',
  18: 'V6',
  19: 'V7',
  20: 'V8',
 

In [8]:
from itertools import permutations
[kk for kk in permutations(grades.columns, 2)]

[('id', 'french'),
 ('id', 'usa'),
 ('id', 'v-bouldering'),
 ('french', 'id'),
 ('french', 'usa'),
 ('french', 'v-bouldering'),
 ('usa', 'id'),
 ('usa', 'french'),
 ('usa', 'v-bouldering'),
 ('v-bouldering', 'id'),
 ('v-bouldering', 'french'),
 ('v-bouldering', 'usa')]

In [ ]:
from climb.grade import import_gradestable

In [ ]:
logs = extract_climbinglogs(filein,
                            cols_ffill=['date', 'climbing_gym', 'grade']
                           )

In [ ]:
logs

### 2. Create a route pyramid

In [ ]:
fig = plot_gradepyramid(logs, aggtype='sum', gradesystem='french')

In [ ]:
fig = plot_doublegradepyramid(logs, aggtype='sum', gradesystem='french')
ax1, ax2 = fig.get_axes()

ax1.annotate("", 
             xy=(11, get_maxgrade_flash(logs[logs['style'] == 'lead'], gradesystem='french')),
             xytext=(12, get_maxgrade_flash(logs[logs['style'] == 'lead'], gradesystem='french')),
             arrowprops=dict(arrowstyle="->", lw=3))
ax2.annotate("", 
             xy=(11, get_maxgrade_flash(logs[logs['style'] == 'toprope'], gradesystem='french')),
             xytext=(12, get_maxgrade_flash(logs[logs['style'] == 'toprope'], gradesystem='french')),
             arrowprops=dict(arrowstyle="->", lw=3))

In [ ]:
temp = logs['grade_french'].sort_values().to_list()
if len(temp) % 2 == 0:
    print(temp[int((len(temp)) / 2)])
else:
    print(temp[int((len(temp) + 1) / 2)])

In [ ]:
logs['week'] = logs['date'].dt.isocalendar().week

In [ ]:
pyrm = (logs[logs['ascension_type'] == 'redpoint']
        .groupby(['week', 'grade_french'])
        .agg(sends=('sends', 'sum'),
            )
       )

temp = (logs[logs['ascension_type'] == 'redpoint']
        .groupby('week')
        .agg(sends_total=('sends', 'sum'),
            )
       )

# Join the two dataframes
pyrm = pyrm.reset_index()
temp = temp.reset_index()

pyrm = pyrm.merge(temp, on=['week'], how='left')

pyrm['sends_percentage'] = pyrm['sends'] / pyrm['sends_total']
pyrm.loc[:, 'sends_percentage'] = pyrm.loc[:, 'sends_percentage'].fillna(0.)

# Perform pivot
pyrm = pyrm.pivot(index='grade_french', columns='week', values='sends_percentage')

pyrm = pyrm.drop(index=['1', '2', '3', 
                        '7c+', '8a', '8a+', '8b',
                       ])

pyrm

import seaborn as sns

fig = plt.figure(figsize=(16,16))
ax = fig.add_subplot(111)
sns.heatmap(pyrm, cmap='Blues', ax=ax)
ax.invert_yaxis()

In [ ]:
pyrm = (logs[logs['ascension_type'] == 'flash']
        .groupby(['date', 'grade_french'])
        .agg(sends=('sends', 'sum'),
            )
       )

temp = (logs[logs['ascension_type'] == 'flash']
        .groupby('date')
        .agg(sends_total=('sends', 'sum'),
            )
       )

# Join the two dataframes
pyrm = pyrm.reset_index()
temp = temp.reset_index()

pyrm = pyrm.merge(temp, on=['date'], how='left')

pyrm['sends_percentage'] = pyrm['sends'] / pyrm['sends_total']
pyrm.loc[:, 'sends_percentage'] = pyrm.loc[:, 'sends_percentage'].fillna(0.)

# Perform pivot
pyrm = pyrm.pivot(index='grade_french', columns='date', values='sends_percentage')

pyrm = pyrm.drop(index=['1', '2', '3', 
                        '7c+', '8a', '8a+', '8b',
                       ])

pyrm

import seaborn as sns

fig = plt.figure(figsize=(16,16))
ax = fig.add_subplot(111)
sns.heatmap(pyrm, cmap='Blues', ax=ax)
ax.invert_yaxis()

In [ ]:
from matplotlib import rc, rcParams
import matplotlib.gridspec as gridspec
from matplotlib.ticker import MultipleLocator, FixedLocator, FixedFormatter
rc('xtick', labelsize=15) 
rc('ytick', labelsize=15) 
rc("lines", markeredgewidth=2.0)
rc("axes", linewidth=2.0)
rc('font', family='serif')
rcParams["font.size"] = 15
rcParams['xtick.direction'] = 'out'
rcParams['ytick.direction'] = 'out'

In [ ]:
pyrm

In [ ]:
fig = plt.figure(figsize=(8,8))
gs = gridspec.GridSpec(1, 8, width_ratios=[1,1,1,1,1,1,1,1])
ax = fig.add_subplot(gs[0,:7])
axcb = fig.add_axes(gs[0,7].get_position(fig))

pos = ax.imshow(pyrm.values, cmap='Blues', interpolation='none', aspect='auto', origin='lower')

cbar = fig.colorbar(pos, cax=axcb, format='%1.1f')
cbar.set_label('Percentage of routes during session')

ax.xaxis.set_major_locator(FixedLocator([ii for ii in range(pyrm.shape[1])]))
ax.yaxis.set_major_locator(FixedLocator([ii for ii in range(pyrm.shape[0])]))
ax.set_xticklabels([val.strftime('%Y-%m-%d') for val in pyrm.columns.to_list()],
                   rotation=90)
ax.set_yticklabels(pyrm.index.to_list())
ax.tick_params(axis='both',which='major',length=10,width=2)

axcb.grid(color='k')
axcb.tick_params(left='on')

In [ ]:
temp

In [ ]:
logs[logs['grade_french'] == '7b+']

In [ ]:
cond = (logs['ascension_type'] != 'not topped') & (logs['style'] != 'boulder') & (logs['style'] != 'autobelay')
temp = logs[cond]
temp = (pd.DataFrame(temp.values.repeat(temp.sends,
                                        axis=0),
                     columns=temp.columns
                    )
        .astype(temp.dtypes)
       )
temp = temp.groupby('week').agg(maxgrade=('grade_french', 'max'),
                                mediangrade=('grade_french', get_mediangrade),
                                mingrade=('grade_french', 'min'),
                               )


import json
with open('./climb/grades.json') as ff:
    grades = json.load(ff)['french2usa']
grade_map = {kk:vv for kk, vv in zip(grades.keys(), range(len(grades)))}

temp['maxgrade_int'] = [grade_map[val] for val in temp['maxgrade']]
temp['mediangrade_int'] = [grade_map[val] for val in temp['mediangrade']]
temp['mingrade_int'] = [grade_map[val] for val in temp['mingrade']]


from matplotlib.ticker import FormatStrFormatter, MultipleLocator

fig = plt.figure(figsize=(16,8))
ax = fig.add_subplot(111)
ax.plot(temp.index, temp['maxgrade_int'], 'k-v')
ax.plot(temp.index, temp['mediangrade_int'], 'k-s')
ax.plot(temp.index, temp['mingrade_int'], 'k-^')
ax.fill_between(temp.index.to_list(), temp['mingrade_int'].values, temp['maxgrade_int'].values, color='k', alpha=.2)

# Determine min and max of the grades plotted
grade_plotmin = temp.loc[:, ['maxgrade', 'mediangrade', 'mingrade']].min().min()
grade_plotmax = temp.loc[:, ['maxgrade', 'mediangrade', 'mingrade']].max().max()

# Determine the list of labels
grade_invmap = {vv:kk for kk, vv in grade_map.items()}
labels = [grade_invmap[vv] for vv in range(grade_map[grade_plotmin] - 2, grade_map[grade_plotmax] + 1, 1)]
# WARNING
# Need to determine why this is -2, instead of -1

ax.set_ylim([grade_map[grade_plotmin] - 1, 
             grade_map[grade_plotmax] + 1
            ])
             
ax.yaxis.set_major_locator(MultipleLocator(1))
ax.set_yticklabels(labels)

ax.xaxis.set_major_locator(MultipleLocator(1))
ax.xaxis.set_major_formatter(FormatStrFormatter('%.0f'))
ax.grid(color='k', alpha=.1)
ax.set_ylabel('Climbing grade (french system)')
ax.set_xlabel('Week number')



In [ ]:
cond = (logs['ascension_type'] != 'not topped') & (logs['style'] != 'boulder') & (logs['style'] != 'autobelay')
temp = logs[cond]
temp = (pd.DataFrame(temp.values.repeat(temp.sends,
                                        axis=0),
                     columns=temp.columns
                    )
        .astype(temp.dtypes)
       )
temp = temp.groupby('week').agg(maxgrade=('grade_french', 'max'),
                                mediangrade=('grade_french', get_mediangrade),
                                mingrade=('grade_french', 'min'),
                               )


import json
with open('./climb/grades.json') as ff:
    grades = json.load(ff)['french2usa']
grade_map = {kk:vv for kk, vv in zip(grades.keys(), range(len(grades)))}

temp['maxgrade_int'] = [grade_map[val] for val in temp['maxgrade']]
temp['mediangrade_int'] = [grade_map[val] for val in temp['mediangrade']]
temp['mingrade_int'] = [grade_map[val] for val in temp['mingrade']]
temp

In [ ]:
cond = (logs['ascension_type'] == 'flash') & (logs['style'] != 'boulder') & (logs['style'] != 'autobelay')
temp = logs[cond]
temp = (pd.DataFrame(temp.values.repeat(temp.sends,
                                        axis=0),
                     columns=temp.columns
                    )
        .astype(temp.dtypes)
       )
temp = temp.groupby('week').agg(maxgrade=('grade_french', 'max'),
                                mediangrade=('grade_french', get_mediangrade),
                                mingrade=('grade_french', 'min'),
                               )


import json
with open('./climb/grades.json') as ff:
    grades = json.load(ff)['french2usa']
grade_map = {kk:vv for kk, vv in zip(grades.keys(), range(len(grades)))}

temp['maxgrade_int'] = [grade_map[val] for val in temp['maxgrade']]
temp['mediangrade_int'] = [grade_map[val] for val in temp['mediangrade']]
temp['mingrade_int'] = [grade_map[val] for val in temp['mingrade']]
temp

In [ ]:
cond = (logs['ascension_type'] == 'redpoint') & (logs['style'] != 'boulder') & (logs['style'] != 'autobelay')
temp = logs[cond]
temp = (pd.DataFrame(temp.values.repeat(temp.sends,
                                        axis=0),
                     columns=temp.columns
                    )
        .astype(temp.dtypes)
       )
temp = temp.groupby('week').agg(maxgrade=('grade_french', 'max'),
                                mediangrade=('grade_french', get_mediangrade),
                                mingrade=('grade_french', 'min'),
                               )


import json
with open('./climb/grades.json') as ff:
    grades = json.load(ff)['french2usa']
grade_map = {kk:vv for kk, vv in zip(grades.keys(), range(len(grades)))}

temp['maxgrade_int'] = [grade_map[val] for val in temp['maxgrade']]
temp['mediangrade_int'] = [grade_map[val] for val in temp['mediangrade']]
temp['mingrade_int'] = [grade_map[val] for val in temp['mingrade']]
temp

In [ ]:
fig = plt.figure(figsize=(16,8))
ax = fig.add_subplot(111)
ax.errorbar(temp.index, temp['mediangrade_int'],
            yerr=np.vstack((temp['mediangrade_int'] - temp['mingrade_int'], temp['maxgrade_int'] - temp['mediangrade_int'])),
            color='k',
            marker='s',
            ecolor='gray',
            capsize=8,
            capthick=2,
            elinewidth=2,
            ms=14
           )

# Determine min and max of the grades plotted
grade_plotmin = temp.loc[:, ['maxgrade', 'mediangrade', 'mingrade']].min().min()
grade_plotmax = temp.loc[:, ['maxgrade', 'mediangrade', 'mingrade']].max().max()

# Determine the list of labels
grade_invmap = {vv:kk for kk, vv in grade_map.items()}
labels = [grade_invmap[vv] for vv in range(grade_map[grade_plotmin] - 1, grade_map[grade_plotmax] + 2, 1)]

ax.set_ylim([grade_map[grade_plotmin] - 1, 
             grade_map[grade_plotmax] + 1
            ])
             
ax.yaxis.set_major_locator(MultipleLocator(1))
ax.set_yticklabels(labels)

ax.xaxis.set_major_locator(MultipleLocator(1))
ax.xaxis.set_major_formatter(FormatStrFormatter('%.0f'))
ax.grid(color='k', alpha=.1)
ax.set_ylabel('Climbing grade (french system)')
ax.set_xlabel('Week number')